In [1]:
import tensorflow as tf
import os
from bert import run_classifier, modeling

In [2]:
model_dir = "./output"
bert_base_dir = "./bert_base"
data_dir = "./data"
bert_config_file = os.path.join(bert_base_dir, "bert_config.json")
init_checkpoint = os.path.join(bert_base_dir, "bert_model.ckpt")
learning_rate = 2e-5
num_train_steps = 12184
num_warmup_steps = 1218
max_seq_length = 128

In [3]:
test_input_fn = run_classifier.file_based_input_fn_builder(
    input_file=os.path.join(data_dir, "test.tf_record"),
    seq_length=max_seq_length,
    is_training=False,
    drop_remainder=False)

In [4]:
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

In [5]:
model_fn = run_classifier.model_fn_builder(bert_config=bert_config,
                                           num_labels=7,
                                           init_checkpoint=init_checkpoint,
                                           learning_rate=learning_rate,
                                           num_train_steps=num_train_steps,
                                           num_warmup_steps=num_warmup_steps,
                                           use_tpu=False,
                                           use_one_hot_embeddings=False)

In [6]:
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    cluster=None,
    master=None,
    model_dir=model_dir,
    save_checkpoints_steps=500,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=50,
        num_shards=8,
        per_host_input_for_training=is_per_host))

In [7]:
estimator = tf.contrib.tpu.TPUEstimator(model_fn=model_fn,
                                        model_dir=model_dir,
                                        config=run_config,
                                        use_tpu=False,
                                        predict_batch_size=8,
                                        eval_batch_size=8,
                                        warm_start_from=os.path.join(
                                            model_dir, "checkpoint"))

INFO:tensorflow:Using config: {'_global_id_in_cluster': 0, '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f401a6160f0>, '_num_ps_replicas': 0, '_model_dir': './output', '_tpu_config': TPUConfig(iterations_per_loop=50, num_shards=8, computation_shape=None, per_host_input_for_training=3, tpu_job_name=None, initial_infeed_sleep_secs=None), '_master': '', '_log_step_count_steps': None, '_session_config': None, '_train_distribute': None, '_cluster': None, '_save_checkpoints_secs': None, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_tf_random_seed': None, '_is_chief': True, '_save_summary_steps': 100, '_task_id': 0, '_service': None, '_keep_checkpoint_max': 5, '_save_checkpoints_steps': 500, '_evaluation_master': '', '_device_fn': None}
INFO:tensorflow:_TPUContext: eval_on_tpu True


In [8]:
result = estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running eval on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 128)
INFO:tensorflow:  name = input_mask, shape = (?, 128)
INFO:tensorflow:  name = is_real_example, shape = (?,)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 128)
INFO:tensorflow:Initialize variable bert/embeddings/LayerNorm/beta:0 from checkpoint ./bert_base/bert_model.ckpt with bert/embeddings/LayerNorm/beta
INFO:tensorflow:Initialize variable bert/embeddings/LayerNorm/gamma:0 from checkpoint ./bert_base/bert_model.ckpt with bert/embeddings/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/embeddings/position_embeddings:0 from checkpoint ./bert_base/bert_model.ckpt with bert/embeddings/position_embeddings
INFO:tensorflow:Initialize variable bert/embeddings/token_type_embeddings:0 from checkpoint ./bert_base/bert_model.ckpt with bert/embeddings/token_type_embeddings
INFO:tens

INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/self/query/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_10/attention/self/query/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/self/value/bias:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_10/attention/self/value/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_10/attention/self/value/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_10/attention/self/value/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_10/intermediate/dense/bias:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_10/intermediate/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_10/intermediate/dense/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_10/intermediate/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_10/output/LayerNorm/beta:0

INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/self/key/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_3/attention/self/key/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/self/query/bias:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_3/attention/self/query/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/self/query/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_3/attention/self/query/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/self/value/bias:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_3/attention/self/value/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_3/attention/self/value/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_3/attention/self/value/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_3/intermediate/dense/bias:0 from

INFO:tensorflow:Initialize variable bert/encoder/layer_6/attention/output/dense/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_6/attention/output/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_6/attention/self/key/bias:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_6/attention/self/key/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_6/attention/self/key/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_6/attention/self/key/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_6/attention/self/query/bias:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_6/attention/self/query/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_6/attention/self/query/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_6/attention/self/query/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_6/attention/self/value/bias:0 fr

INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/output/LayerNorm/gamma:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_9/attention/output/LayerNorm/gamma
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/output/dense/bias:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_9/attention/output/dense/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/output/dense/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_9/attention/output/dense/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/key/bias:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_9/attention/self/key/bias
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/key/kernel:0 from checkpoint ./bert_base/bert_model.ckpt with bert/encoder/layer_9/attention/self/key/kernel
INFO:tensorflow:Initialize variable bert/encoder/layer_9/attention/self/q

INFO:tensorflow:  name = bert/encoder/layer_2/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_2/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_3/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder

INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_7/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflo

INFO:tensorflow:  name = bert/pooler/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = output_weights:0, shape = (7, 768)
INFO:tensorflow:  name = output_bias:0, shape = (7,)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-08-15-07:14:44
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./output/model.ckpt-12184
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-08-15-07:17:34
INFO:tensorflow:Saving dict for global step 12184: eval_accuracy = 0.0720862, eval_loss = 10.2124405, global_step = 12184, loss = 10.212654
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 12184: ./output/model.ckpt-12184


In [10]:
output_eval_file = os.path.join(model_dir, "eval_results.txt")
with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

INFO:tensorflow:***** Eval results *****
INFO:tensorflow:  eval_accuracy = 0.0720862
INFO:tensorflow:  eval_loss = 10.2124405
INFO:tensorflow:  global_step = 12184
INFO:tensorflow:  loss = 10.212654
